# Preparing

## Imports and Functions

In [76]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import regex as re
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

### Normal

In [3]:
folder_path = '/Users/Felix/Google Drive/Felix/code/AgeRecognition/raw_data/Faces'
#folder_path = '/content/UTKFaces/UTKFaces/Faces'

In [19]:
def load_images_from_folder(folder_path):
    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    images = []
    for filename in filenames:
        img = cv2.imread(os.path.join(folder_path,filename),cv2.IMREAD_UNCHANGED)
        
        scale_percent = 50 # percent of original size
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)
        
        # resize image
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        cv2.imshow("Resized image", resized)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        if img is not None:
            img_colors = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
            images.append(img_colors)
    return images

def show_all_images_from_list(img_list,img_df):
    n=int(len(img_list)/5) +1 
    f = plt.figure(figsize=(25,n*5))
    for i,img in enumerate(img_list):
        f.add_subplot(n, 5, i + 1)
        plt.title(f'{img_df.iloc[i]}',
                  fontweight="bold",
                  fontsize=15)
        plt.imshow(img)
        f.tight_layout()

def load_image_data_into_dataframe(folderpath):

    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    df = pd.DataFrame()
    for filename in filenames:
        if filename.startswith('81'):
            break
        parsed_name = filename.split('_')
        df= df.append({'age' : int(parsed_name[0]),
                       'gender' : parsed_name[1],
                       'race' : parsed_name[2]}, ignore_index=True)
    df['gender'] = df['gender'].map({'0': 'male', '1': 'female'})
    df['race'] = df['race'].map({'0': 'White', '1': 'Black', '2': 'Asian', '3': 'Indian', '4': 'Others'})
    return df

In [20]:
img_pic=load_images_from_folder(folder_path)

In [11]:
img_df = load_image_data_into_dataframe(folder_path)

## Targets into Age Bins (Baseline: 10%)

In [13]:
#bins_value=[1,5,19,25,28,32,39,49,58,80]
a=pd.qcut(img_df['age'],10)

In [14]:
a.value_counts().sort_index()

(0.999, 5.0]    2308
(5.0, 19.0]     2120
(19.0, 25.0]    2795
(25.0, 26.0]    2031
(26.0, 29.0]    1938
(29.0, 34.0]    2193
(34.0, 39.0]    2168
(39.0, 49.0]    2165
(49.0, 58.0]    2124
(58.0, 80.0]    2163
Name: age, dtype: int64

In [15]:
img_df['age_group'] = pd.qcut(img_df['age'],10).astype(str)
img_df['age_group'] = img_df['age_group'].map({
    '(0.999, 5.0]' : 0,
    '(5.0, 19.0]' : 1,
    '(19.0, 25.0]': 2,
    '(25.0, 26.0]': 3,
    '(26.0, 29.0]':4,
    '(29.0, 34.0]':5,
    '(34.0, 39.0]':6,
    '(39.0, 49.0]':7,
    '(49.0, 58.0]':8,
    '(58.0, 80.0]':9})

## CNN 1 Preperations (Size: Original)

In [13]:
X = img_pic
y = img_df['age_group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test =np.array(y_test)
X_train.shape

In [15]:
X_train = X_train/255 - 0.5
X_test = X_test/255 - 0.5
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [16]:
#np.save('X_train', X_train)
#np.save('X_test', X_test)
#np.save('y_train_cat', y_train_cat)
#np.save('y_test_cat', y_test_cat)

In [20]:
print(X_train.shape)
print(X_test.shape)

(19804, 200, 200, 3)
(2201, 200, 200, 3)


## CNN 1 (Accuracy: 12,59 %)

In [28]:
def initialize_model():
    model = models.Sequential()
    ### First convolution & max-pooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(200, 200, 3),
                            activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One fully connected
    model.add(layers.Dense(10, activation='relu'))

    ### Last layer (let's say a classification with 10 output)
    model.add(layers.Dense(10, activation='softmax'))
    
    ### Model compilation
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [22]:
model = initialize_model()

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

Epoch 1/10
434/434 [==============================] - 176s 403ms/step - loss: 2.3169 - accuracy: 0.1197 - val_loss: 2.2984 - val_accuracy: 0.1355
Epoch 2/10
434/434 [==============================] - 120s 276ms/step - loss: 2.2996 - accuracy: 0.1236 - val_loss: 2.2968 - val_accuracy: 0.1355
Epoch 3/10
434/434 [==============================] - 134s 309ms/step - loss: 2.2992 - accuracy: 0.1236 - val_loss: 2.2962 - val_accuracy: 0.1355
Epoch 4/10
434/434 [==============================] - 127s 293ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2961 - val_accuracy: 0.1355
Epoch 5/10
434/434 [==============================] - 112s 258ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2960 - val_accuracy: 0.1355
Epoch 6/10
434/434 [==============================] - 149s 344ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2960 - val_accuracy: 0.1355
Epoch 7/10
434/434 [==============================] - 148s 341ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2959 -

In [24]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

0.12585188448429108


## CNN Preperations Image Size: 100*100

In [22]:
X = img_pic
y = img_df['age_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test =np.array(y_test)
X_train.shape

In [24]:
X_train = X_train/255 - 0.5
X_test = X_test/255 - 0.5
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [27]:
print(X_train.shape)
print(X_test.shape)

(19804, 100, 100, 3)
(2201, 100, 100, 3)


# CNN A

## CNN 2 (1 hidden with 10) 12,59%

In [30]:
def initialize_model():
    model = models.Sequential()
    ### First convolution & max-pooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                            activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One fully connected
    model.add(layers.Dense(10, activation='relu'))

    ### Last layer (let's say a classification with 10 output)
    model.add(layers.Dense(10, activation='softmax'))
    
    ### Model compilation
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [32]:
model2 = initialize_model()

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model2.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

Epoch 1/10
434/434 [==============================] - 25s 57ms/step - loss: 2.3021 - accuracy: 0.1232 - val_loss: 2.2979 - val_accuracy: 0.1355
Epoch 2/10
434/434 [==============================] - 22s 51ms/step - loss: 2.2994 - accuracy: 0.1236 - val_loss: 2.2966 - val_accuracy: 0.1355
Epoch 3/10
434/434 [==============================] - 22s 51ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2961 - val_accuracy: 0.1355
Epoch 4/10
434/434 [==============================] - 23s 52ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2960 - val_accuracy: 0.1355
Epoch 5/10
434/434 [==============================] - 23s 52ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2959 - val_accuracy: 0.1355
Epoch 6/10
434/434 [==============================] - 23s 53ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2959 - val_accuracy: 0.1355
Epoch 7/10
434/434 [==============================] - 23s 52ms/step - loss: 2.2991 - accuracy: 0.1236 - val_loss: 2.2959 - val_accuracy:

In [33]:
print(model2.evaluate(X_test, y_test_cat, verbose=0)[1])

0.12585188448429108


## CNN 3 (1 hidden with 50 ) 40,25 %

In [61]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_32 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_82 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_83 (Dense)             (None, 10)                510       
Total params: 922,552
Trainable params: 922,552
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 31s 70ms/step - loss: 1.9107 - accuracy: 0.2824 - val_

## CNN 4 (1 hidden with 100 )  40,75%

In [62]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_84 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_85 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 28s 64ms/step - loss: 1.8635 - accuracy: 0.3078 - 

## CNN 5 (2 hidden with 100,50 )  39,7%

In [50]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
434/434 [==============================] - 39s 90ms/step - loss: 1.8448 - accuracy: 0.3131 - val_loss: 1.6139 - val_accuracy: 0.3785
Epoch 2/10
434/434 [==============================] - 30s 68ms/step - loss: 1.5327 - accuracy: 0.4123 - val_loss: 1.5671 - val_accuracy: 0.3963
Epoch 3/10
434/434 [==============================] - 28s 64ms/step - loss: 1.4061 - accuracy: 0.4592 - val_loss: 1.5753 - val_accuracy: 0.3958
Epoch 4/10
434/434 [==============================] - 35s 81ms/step - loss: 1.2902 - accuracy: 0.5039 - val_loss: 1.5731 - val_accuracy: 0.4078
0.39709222316741943


## CNN 6 (2 hidden with 100,100 ) 39,3%

In [51]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
434/434 [==============================] - 38s 86ms/step - loss: 1.8797 - accuracy: 0.3016 - val_loss: 1.6689 - val_accuracy: 0.3716
Epoch 2/10
434/434 [==============================] - 41s 95ms/step - loss: 1.5824 - accuracy: 0.3960 - val_loss: 1.5832 - val_accuracy: 0.3886
Epoch 3/10
434/434 [==============================] - 43s 99ms/step - loss: 1.4627 - accuracy: 0.4377 - val_loss: 1.5788 - val_accuracy: 0.3928
Epoch 4/10
434/434 [==============================] - 37s 86ms/step - loss: 1.3620 - accuracy: 0.4750 - val_loss: 1.5654 - val_accuracy: 0.3901
Epoch 5/10
434/434 [==============================] - 35s 80ms/step - loss: 1.2679 - accuracy: 0.5154 - val_loss: 1.6142 - val_accuracy: 0.3906
Epoch 6/10
434/434 [==============================] - 35s 81ms/step - loss: 1.1718 - accuracy: 0.5511 - val_loss: 1.6562 - val_accuracy: 0.4026
0.3930031657218933


## CNN 7 (3 hidden with 50,100,50 )  40,3%

In [63]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_34 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_86 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_87 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_88 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_89 (Dense)             (None, 10)              

## CNN 8 (1 hidden with 300 )  39,7%

In [56]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(300, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_67 (Dense)             (None, 300)               5529900   
_________________________________________________________________
dense_68 (Dense)             (None, 10)                3010      
Total params: 5,533,302
Trainable params: 5,533,302
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 50s 108ms/step - loss: 1.8050 - accuracy: 0.3240 -

## CNN 9 (2 hidden with 100,20 )  39,6%

In [57]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_69 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_70 (Dense)             (None, 20)                2020      
_________________________________________________________________
dense_71 (Dense)             (None, 10)                210       
Total params: 1,845,922
Trainable params: 1,845,922
Non-trainable params: 0
___________________________________________

## CNN 10 (3 hidden with 20, 50, 20 )  ???%

In [65]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(20, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Third fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_36 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_92 (Dense)             (None, 20)                368660    
_________________________________________________________________
dense_93 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_94 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_95 (Dense)             (None, 10)              

# CNN B

## CNN 1 (1 hidden with 100) - (Epochs:100, Patience: 10)   41,3%
**High_Acc - low Val_Acc**

In [64]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_35 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_90 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_91 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
434/434 [==============================] - 87s 200ms/step - loss: 1.8293 - accuracy: 0.3153 

## CNN 1.2 - Dropout, Hidden Layer 50% => 41,6%

In [68]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.5))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_38 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_97 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

## CNN 1.3 Dropout First and Hidden Layer 20%    34,6 %

In [69]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(Dropout(0.2, input_shape=(100,100,3,)))
model.add(layers.Conv2D(8, (4,4), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_39 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_99 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_100 (Dense)            (None, 10)              

## CNN 1.4 Dropout, Hidden Layer 40% => 40,4%

In [70]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.4))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_40 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_101 (Dense)            (None, 100)               1843300   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_102 (Dense)            (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

## CNN 1.5 Dropout, Hidden Layer 30% => 39,9%

In [82]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.3))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_47 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_115 (Dense)            (None, 100)               1843300   
_________________________________________________________________
dropout_16 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_116 (Dense)            (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

## CNN 1.6 Dropout, Hidden Layer 20% => 41,4%

In [77]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_43 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_107 (Dense)            (None, 100)               1843300   
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_108 (Dense)            (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

KeyboardInterrupt: 

## CNN 2 Something new (Copy from Cifar Classification) 42,2 %

In [79]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 25, 25, 128)     

In [80]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

0.4220808744430542
